## Polymer-aware Feature Engineering

This notebook constructs polymer-aware, repeat-unit-level features to augment
generic RDKit molecular descriptors. The goal is to incorporate domain-specific
polymer structural information (composition, rigidity, flexibility) and evaluate
their impact on Tg prediction using the existing baseline models.

In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

In [19]:
df = pd.read_csv('../data/intermediate/tg_with_rdkit_descriptors.csv')

In [7]:
def atom_fraction_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    atoms = [atom.GetSymbol() for atom in mol.GetAtoms()]
    total = mol.GetNumAtoms()
    def frac(x):
        return atoms.count(x) / total if total > 0 else 0

    return {
    "frac_C": frac("C"),
    "frac_O": frac("O"),
    "frac_N": frac("N"),
    "frac_S": frac("S"),
    "frac_hetero": 1 - frac("C")
    }

In [8]:
def rigidity_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    aromatic_atoms = sum(a.GetIsAromatic() for a in mol.GetAtoms())
    
    return {
        "aromatic_ring_count": rdMolDescriptors.CalcNumAromaticRings(mol),
        "aromatic_atom_frac": aromatic_atoms / mol.GetNumAtoms(),
        "double_bond_count": sum(
            b.GetBondTypeAsDouble() == 2 for b in mol.GetBonds()
        )
    }

In [9]:
def flexibility_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    num_bonds = mol.GetNumBonds()
    
    return {
        "rotatable_bond_frac": (
            rdMolDescriptors.CalcNumRotatableBonds(mol) / num_bonds
            if num_bonds > 0 else 0
        ),
        "heavy_atom_count": mol.GetNumHeavyAtoms()
    }

In [10]:
def polymer_aware_features(smiles):
    features = {}

    for func in [
        atom_fraction_features,
        rigidity_features,
        flexibility_features
    ]:
        out = func(smiles)
        if out is not None:
            features.update(out)

    return features

In [20]:
df.head()

,SMILES,Tg,PID,Polymer Class,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,*C*,-54.0,P010001,Polyolefins,1.750000,1.750000,0.875000,0.875000,0.355446,20.000000,...,0,0,0,0,0,0,0,0,0,0
1,*CC(*)C,-3.0,P010002,Polyolefins,2.395833,2.395833,0.750000,0.750000,0.414720,25.666667,...,0,0,0,0,0,0,0,0,0,0
2,*CC(*)CC,-24.1,P010003,Polyolefins,2.332824,2.332824,0.743056,0.743056,0.451401,22.250000,...,0,0,0,0,0,0,0,0,0,0
3,*CC(*)CCC,-37.0,P010004,Polyolefins,2.310000,2.310000,0.734306,0.734306,0.476641,20.200000,...,0,0,0,0,0,0,0,0,0,0
4,*CC(*)C(C)C,60.0,P010006,Polyolefins,2.374491,2.374491,0.699074,0.699074,0.465496,21.400000,...,0,0,0,0,0,0,0,0,0,0


In [21]:
poly_feat_series = df["SMILES"].apply(polymer_aware_features)
poly_feat_df = pd.DataFrame(list(poly_feat_series))

In [22]:
poly_feat_df.head()

,frac_C,frac_O,frac_N,frac_S,frac_hetero,aromatic_ring_count,aromatic_atom_frac,double_bond_count,rotatable_bond_frac,heavy_atom_count
0,0.333333,0.0,0.0,0.0,0.666667,0,0.0,0,0.000000,1
1,0.600000,0.0,0.0,0.0,0.400000,0,0.0,0,0.250000,3
2,0.666667,0.0,0.0,0.0,0.333333,0,0.0,0,0.400000,4
3,0.714286,0.0,0.0,0.0,0.285714,0,0.0,0,0.500000,5
4,0.714286,0.0,0.0,0.0,0.285714,0,0.0,0,0.333333,5


In [23]:
poly_feat_df.isna().mean().sort_values(ascending=False)

frac_C                 0.0
frac_O                 0.0
frac_N                 0.0
frac_S                 0.0
frac_hetero            0.0
aromatic_ring_count    0.0
aromatic_atom_frac     0.0
double_bond_count      0.0
rotatable_bond_frac    0.0
heavy_atom_count       0.0
dtype: float64

In [24]:
poly_feat_df.describe().T

,count,mean,std,min,25%,50%,75%,max
frac_C,7284.0,0.725277,0.111612,0.00000,0.681818,0.744186,0.795918,0.975610
frac_O,7284.0,0.117301,0.068669,0.00000,0.076923,0.114286,0.153846,0.437500
frac_N,7284.0,0.039123,0.042492,0.00000,0.000000,0.034483,0.062500,0.444444
frac_S,7284.0,0.006234,0.022229,0.00000,0.000000,0.000000,0.000000,0.500000
frac_hetero,7284.0,0.274723,0.111612,0.02439,0.204082,0.255814,0.318182,1.000000
aromatic_ring_count,7284.0,3.245607,2.763247,0.00000,1.000000,3.000000,5.000000,17.000000
aromatic_atom_frac,7284.0,0.439524,0.265559,0.00000,0.292683,0.480000,0.642857,0.965517
double_bond_count,7284.0,2.565074,2.026535,0.00000,1.000000,2.000000,4.000000,16.000000
rotatable_bond_frac,7284.0,0.269242,0.160588,0.00000,0.150000,0.214286,0.358279,0.927273
heavy_atom_count,7284.0,34.464992,18.971524,1.00000,19.000000,33.000000,46.000000,164.000000


In [25]:
tg_descriptors_polymer_features = pd.concat([df.reset_index(drop=True), poly_feat_df.reset_index(drop=True)], axis=1)

In [29]:
tg_descriptors_polymer_features.head()

,SMILES,Tg,PID,Polymer Class,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,...,frac_C,frac_O,frac_N,frac_S,frac_hetero,aromatic_ring_count,aromatic_atom_frac,double_bond_count,rotatable_bond_frac,heavy_atom_count
0,*C*,-54.0,P010001,Polyolefins,1.750000,1.750000,0.875000,0.875000,0.355446,20.000000,...,0.333333,0.0,0.0,0.0,0.666667,0,0.0,0,0.000000,1
1,*CC(*)C,-3.0,P010002,Polyolefins,2.395833,2.395833,0.750000,0.750000,0.414720,25.666667,...,0.600000,0.0,0.0,0.0,0.400000,0,0.0,0,0.250000,3
2,*CC(*)CC,-24.1,P010003,Polyolefins,2.332824,2.332824,0.743056,0.743056,0.451401,22.250000,...,0.666667,0.0,0.0,0.0,0.333333,0,0.0,0,0.400000,4
3,*CC(*)CCC,-37.0,P010004,Polyolefins,2.310000,2.310000,0.734306,0.734306,0.476641,20.200000,...,0.714286,0.0,0.0,0.0,0.285714,0,0.0,0,0.500000,5
4,*CC(*)C(C)C,60.0,P010006,Polyolefins,2.374491,2.374491,0.699074,0.699074,0.465496,21.400000,...,0.714286,0.0,0.0,0.0,0.285714,0,0.0,0,0.333333,5


In [32]:
tg_descriptors_polymer_features.shape

(7284, 231)

In [30]:
tg_descriptors_polymer_features.to_csv('../data/intermediate/tg_with_rdkit_and_polymer_features.csv', index=False)